访问外部硬件有两个方式

1. 将某个外设的内存映射到一定范围的地址空间中，CPU通过地址总线访问该内存区域时会落到外设的内存中，这种映射让CPU访问外设的内存就如同访问主板上的物理内存一样。有的设备是这样做的，比如显卡，显卡是显示器的适配器，CPU不直接和显示器交互，它只和显卡通信。显卡上有篇内存叫显存，它被映射到主机物理内存上的地段1MB的0xB800~0xBFFFF。CPU访问这片内存就是访问显存，往这片内存上写字节便是往屏幕上打印内容。

2. 外设通过IO接口与CPU通信的，CPU访问外设，就是访问IO接口，由IO接口将信息传递给给另一端的外设，也就是说，CPU从来不知道有这些设备的存在，它只知道自己操作的IO接口。

编译器提供了一套库函数，库函数中又有封装的系统调用，这样的代码集合称之为运行库。

我们所写的应用程序只是半成品，需要调用操作系统提供好的函数才能完整地做成一件事，而这个函数便是系统调用。

用户态和内核态是对CPU来讲的，是指CPU运行在用户态还是内核态。用户进程陷入内核态是指：由于内部或外部中断发生，当前进程被暂时中止运行，其上下文被内核终端程序保护起来后，开始执行一段内核的代码。

当用户程序陷入内核后，它自己已经下CPU了，以后的事情，应用程序永远不知道，它的上下文环境已经被保存到自己的0特权级栈了，那时在CPU上运行的程序已经是内核程序了。

应用程序处于特权级3，操作系统内核处于特权级0。党用户程序欲访问系统资源时（无论是硬件，还是内核数据结构），它需要进行系统调用。这样CPU进入了内核态，也称管态

将数据和代码分开的好处有三点

1. 可以为他们赋予不同的属性
例如数据本生是需要修改的，所有数据就是需要有读写的属性。代码段需要具备只读的属性
2. 为了提高CPU内部缓存的命中率
3. 节省内存
程序中存在一些制度的部分，比如代码，当一个程序的多个副本同时运行时候，没必要在内存中同时存在多个相同过的代码段，这将浪费有限的物理内存资源，只要把这一代码段共享就可以了。

在保护模式下，有这样一个数据结构，它叫全局描述符表`(Global Descriptor Table)`，这个表中的每一项称为段描述符。

1. 编译器负责挑选出数据具备的属性，从而根据属性将程序片段分类，比如，划分出只读属性的代码段和可写属性的数据段。再补充一下，编译器并没有让段具备某种属性，对于代码段，编译器所作的只是将程序中的有关代码的多个`section`合并成一个大的`segment`,它并没有为代码段添加额外的信息。

2. 操作系统通过设置GDT全局表舒服来构建段描述符，在段描述符中指定段的位置、大小及属性。也就是说，操作系统认为代码应该是只读的，所以给用来指向代码段的那个段描述符设置了只读的属性，这才是真正给段添加属性的地方

3. CPU的段寄存器提前被操作系统赋予了相应的段选择子，从而制定了指向的段。在执行命令时，会更具该段的属性来判断执行的行为，若有返回则发出异常

在实模式下，“段基址+段内偏移地址”经过段部件的处理，直接输出的就是物理地址，CPU可以直接用此地址访问内存。

而在保护模式下，“段基址+段内偏移地址”称为线性地址。不过，此时的段基址已经不是真正的地址了，而是一个称为选择子的东西。它本质是一个索引，类似于数组下标，通过这个索引便能在GDT中找到相应的段描述符，在该描述符中记录了该段的其实、大小等信息，这样便得到了段基址。若没有开启地址分页功能，此线性地址就被当作物理地址来用，可直接访问内存。若开启了分页功能，此线性地址又多了一个名字，就是虚拟地址（虚拟地址、线性地址在分页机制下都是一回事）。虚拟地址要经过CPU页部件转换成具体的物理地址，这样CPU才能将其送上地址总线去访问内存。

无论在实模式下还是在保护模式下，段内偏移地址又称为有效地址，也称为逻辑地址，这是程序员可见的地址。这是因为，最终的地址是由段基址和段内偏移地址组合而成的。由于段基址已经有默认的啦，要么在实模式下的默认段寄存器中，要么是在保护模式下的默认段选择子指向的段描述符中，所以只要给出段内偏移地址就行了，这个地址虽然只是段内偏移，但加上默认的段基址，依然足够有效。

In [ ]:
堆是程序运行过程中用于动态分配的内存空间，是操作系统为每个用户进程规划的，属于